# Sistemas de Recomendación - Netflix Prize Challenge

En este notebook vamos a implementar un sistema de recomendación a través de un filtro colaborativo.

Algunas referencias útiles, además de las mencionadas en la presentación:
* https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system
* https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101

## 1. Carga de Datos y preparación del Dataset

Vamos a empezar cargando uno de los archivos con calificaciones para explorarlo. Como son archivos grandes y van a ocupar bastante lugar en memoria, no vamos a cargar la última columna con fechas.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector

In [2]:
# Load the dataset
df1_20first = pd.read_csv('./datasets/kaggle/recsys/netflix-prize-data/combined_data_1.txt', nrows=20)
df1_20first

,,1:
1488844,3,2005-09-06
822109,5,2005-05-13
885013,4,2005-10-19
30878,4,2005-12-26
823519,3,2004-05-03
893988,3,2005-11-17
124105,4,2004-08-05
1248029,3,2004-04-22
1842128,4,2004-05-09
2238063,3,2005-05-11


In [3]:
def load_data(name):
    df = pd.read_csv(name, header = None, names = ['User','Rating'], usecols = [0,1])
    
    # A veces forzar un tipo de dato hace que se ahorre mucho lugar en memoria.
    df['Rating'] = df['Rating'].astype(float) 
    return df


df1 = load_data('./datasets/kaggle/recsys/netflix-prize-data/combined_data_1.txt')
print(df1.shape)

(24058263, 2)


In [4]:
df1.head()

,User,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


¿Cómo sabemos a qué película corresponde cada calificación? Contar cuántas películas hay en `df1` e identificarlas. Para ello, cargamos `movie_titles.csv`. Como no nos interesa el año, no lo traemos.

In [5]:
df_title = pd.read_csv('./datasets/kaggle/recsys/netflix-prize-data/movie_titles.csv', encoding = "ISO-8859-1",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])
df_title.head()

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


De esta forma, podemos obtener el nombre de una película dado su Id

In [25]:
movie_id = 1
print(df_title.loc[movie_id].Name)

Dinosaur Planet


Para contar cuántos identificadores hay, vamos a usar la siguiente información: al lado del identificador de la película, la columna `Rating` de `df1` tiene un `NaN`.

In [26]:
movies_ids_df1 = df1.User[df1.Rating.isna()].values
print(movies_ids_df1)
print(len(movies_ids_df1))

['1:' '2:' '3:' ... '4497:' '4498:' '4499:']
4499


¿En qué formato está? Si queremos usarlo para pasar de identificador al nombre, debemos llevarlo a enteros. Asumimos que no hay ningun repetido:

In [27]:
movies_ids_df1 = np.arange(1,len(movies_ids_df1) + 1)
print(movies_ids_df1)

[   1    2    3 ... 4497 4498 4499]


### Movie Id

Intentaremos agregar una columna al Dataframe con el Id de la película a la que corresponde la calificación. Para ello, vamos a necesitar saber dónde están ubicados los identificadores.

Primero, seleccionamos los índices donde aparecen los movies_ids

In [28]:
df1_nan = pd.DataFrame(pd.isnull(df1.Rating))
df1_nan = df1_nan[df1_nan['Rating'] == True]
idx_movies_ids = df1_nan.index.values
print(idx_movies_ids)

[       0      548      694 ... 24056849 24057564 24057834]


Queremos crear un vector de tantas instancias como df1, donde en cada lugar esté movie_id a cual corresponde la calificación. Como tenemos los índices donde está cada movie_id, podemos obtener cuántas calificaciones hay de cada película.

In [29]:
# Agregamos el indice de la ultima instancia del dataframe
idx_movies_ids = np.append(idx_movies_ids,df1.shape[0])
cantidad_criticas = np.diff(idx_movies_ids)
cantidad_criticas

array([ 548,  146, 2013, ...,  715,  270,  429])

In [30]:
columna_movie_id = np.array([])
for i in range(cantidad_criticas.size):
    aux = np.full(cantidad_criticas[i], movies_ids_df1[i])
    columna_movie_id = np.concatenate((columna_movie_id, aux))

Agregamos esa columna al dataset

In [31]:
df1['movie_id'] = columna_movie_id
del columna_movie_id

df1.dropna(inplace = True)
df1['User'] = df1['User'].astype(int)
df1['movie_id'] = df1['movie_id'].astype(np.int16)
df1['Rating'] = df1['Rating'].astype(np.int8)

gc.collect()

165

In [32]:
df1

,User,Rating,movie_id
1,1488844,3,1
2,822109,5,1
3,885013,4,1
4,30878,4,1
5,823519,3,1
...,...,...,...
24058258,2591364,2,4499
24058259,1791000,2,4499
24058260,512536,5,4499
24058261,988963,3,4499


In [33]:
df1[df1['Rating'].isna()]

,User,Rating,movie_id


Ya contamos con un dataframe con calificaciones de usuarios a películas.

Una opción es guardar el dataset modificado en nuevo archivo y, a partir de ahora, trabajar con esa versión. Esto hará que no tengamos que hacer el preprocesamiento cada vez que empecemos a trabajar y, además, ahorrarnos toda la "basura" que Python pueda ir dejando en la RAM.

**Ejercicio**: guardar el dataset modificado en un nuevo archivo.

In [ ]:
if False:
    pass
    #completar

In [20]:
df1.to_csv('./datasets/kaggle/recsys/netflix-prize-data/out.csv', index=False)

## 2. Exploración del Dataset

Responder las siguientes preguntas, siempre que se pueda con un lindo gráfico (¡pensar bien cómo!):

1. ¿Cuántos usuarios únicos hay?
2. ¿Cuántas películas calificó cada usuario?
3. ¿Cómo es la distribución de las calificaciones?¿Pueden concluir algo de ese gráfico?
4. ¿Cuál es la película con más calificaciones?¿Cuántas tiene?¿Y la que menos calificaciones tiene?

In [ ]:
# COMPLETAR

### Opcional: filtrar películas con pocos ratings

In [ ]:
# COMPLETAR

## 3. Entrenamiento

Para entrenar el sistema de recomendación vamos a usar la biblioteca Surprise. Recomendamos tener abierta la [documentación](https://surprise.readthedocs.io/en/stable/getting_started.html) a medida que van a través de este notebook.

### 3.1 Dataset y Train/test split

Primero, llevamos el dataset al formato que le gusta a la biblioteca. ¿En qué orden tienen que estar los atributos?. Investigar qué hace la clase `Reader` y cuáles son sus parámetros.

In [22]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-macosx_11_0_arm64.whl size=485277 sha256=7ce38ab0ce7989a2f441281dc96d1cf457d70c0feb71ba349a6ae8851867d517
  Stored in directory: /Users/montse/Library/Caches/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [23]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [24]:
reader = Reader()

Luego, creamos el `Dataset` de Surprise usando `Dataset.load_from_df`

In [25]:
N_filas = 1000000 # Limitamos el dataset a N_filas

data = Dataset.load_from_df(df1[['User','movie_id','Rating'][:N_filas]], reader)

¿Cómo les parece que es mejor hacer el split?¿Dejando películas en test, usuarios o combinaciones?

In [26]:
trainset, testset = train_test_split(data, test_size=0.25)

### 3.2 Entrenamiento

Vamos a entrenar un algoritmo SVD. Explorar sus parámetros y su funcionamiento.

In [27]:
from surprise import SVD
algo = SVD()

Entrenamos sobre el `trainset`

In [28]:
algo.fit(trainset)

Y predecimos sobre el `testset`. Notar que para predecir sobre un conjunto de test se usa la función `test`.

In [29]:
predictions = algo.test(testset)

Explorar las característica de `predictions` y alguno de sus elementos

In [30]:
predictions[1]

Prediction(uid=1134057, iid=2862, r_ui=4.0, est=4.238699027444307, details={'was_impossible': False})

En cambio, si queremos predecir para un usuario y una película en particular, usamos la función `predict`

In [31]:
algo.predict(237993,175)

Prediction(uid=237993, iid=175, r_ui=None, est=4.668726678001573, details={'was_impossible': False})

Exploremos un usuario, veamos cuáles películas le gustaron y cuáles les recomienda el sistema.

Priomero, buscamos qué películas le gustaron

In [32]:
usuario = 237993
rating = 5   # le pedimos peliculas a las que haya puesto 4 o 5 estrellas
df_user = df1[(df1['User'] == 237993) & (df1['Rating'] >= 4)]
df_user = df_user.reset_index(drop=True)
df_user['Name'] = df_title['Name'].loc[df_user.movie_id].values
df_user

,User,Rating,movie_id,Name
0,237993,5,175,Reservoir Dogs
1,237993,4,312,High Fidelity
2,237993,5,424,Happiness
3,237993,5,468,The Matrix: Revolutions
4,237993,5,571,American Beauty
5,237993,4,674,Hellbound: Hellraiser II
6,237993,4,819,The Faculty
7,237993,4,900,Eat Drink Man Woman
8,237993,4,1032,Hard Boiled
9,237993,5,1066,Superman: The Movie


Creamos donde vamos a guardar las recomendaciones

In [33]:
recomendaciones_usuario = df_title.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(4499, 1)


,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


Sacamos del dataframe todas las películas que ya sabemos que vio

In [34]:
usuario_vistas = df1[(df1['User'] == 237993)]
print(usuario_vistas.shape)

(60, 3)


In [37]:
usuario_vistas.head()

,User,Rating,movie_id
14110,237993,3,8
481395,237993,3,143
632214,237993,5,175
1537728,237993,4,312
2244424,237993,5,424


In [38]:
usuario_vistas.index

Index([   14110,   481395,   632214,  1537728,  2244424,  2535860,  3107560,
        3527871,  3878422,  4283895,  4650106,  4991350,  5102985,  5239111,
        5426932,  6813573,  7053849,  7279829,  7445090,  7467147,  7842000,
        8116888,  8391527,  9268981, 10494671, 10760286, 10875895, 10947216,
       11771912, 12115866, 12284851, 12672717, 13623758, 13881392, 13944781,
       14724592, 14821266, 15998861, 16147242, 16289510, 16436226, 16636600,
       17089103, 17722868, 18437618, 19027764, 19472844, 19807143, 20259543,
       20551858, 20838906, 21725259, 21767237, 22809611, 23113844, 23376749,
       23434456, 23489072, 23571269, 23743767],
      dtype='int64')

In [40]:
recomendaciones_usuario

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW
...,...
4495,Clifford: Happy Birthday Clifford / Puppy Love
4496,Farewell My Concubine
4497,Texasville


In [41]:
recomendaciones_usuario.drop(usuario_vistas['movie_id'], inplace = True)
recomendaciones_usuario = recomendaciones_usuario.reset_index()

Y hacemos las recomendaciones

In [42]:
recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['Movie_Id'].apply(lambda x: algo.predict(usuario, x).est)

In [43]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

      Movie_Id                                      Name  Estimate_Score
1227      1243                 La Femme Nikita: Season 2             5.0
3989      4041                   Mobile Suit Gundam SEED             5.0
3621      3668                        Farscape: Season 2             5.0
2031      2057        Buffy the Vampire Slayer: Season 6             5.0
424        430                                   Chobits             5.0
764        774                        Foyle's War: Set 2             5.0
31          33            Aqua Teen Hunger Force: Vol. 1             5.0
1823      1848  Samurai Trilogy 3: Duel at Ganryu Island             5.0
2133      2162                             CSI: Season 1             5.0
923        935                               Read Or Die             5.0


### 3.3 Evaluación

Para el conjunto de `testset`, evaluamos el error RMSE entre las predicciones y las verdaderas calificaciones que le habían dado a las películas. Para eso, buscar en la documentación cómo se hace.

In [44]:
# from surprise import COMPLETAR
from surprise import accuracy

# Calculate RMSE for test dataset
accuracy.rmse(predictions)

RMSE: 0.9029


0.9028931289746546

La próxima clase continuaremos con Optimización de Parámetros y algunos comentarios más sobre cómo trabajar con datasets grandes.